<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Explicit Formulation

$$p_{i}^{n+1} = p^i_n + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot q_{lsc_i} + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot [T_{lx_{i+1/2}}^{n} p_{i+1}^{n}-(T_{lx_{i+1/2}}^{n} + T_{lx_{i-1/2}}^{n})p_{i}^{n}+T_{lx_{i-1/2}}p_{i-1}^{n}]$$

In the following codes, $(\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i$ will be written as `factor`

Transmissibility of coupling cells, $T_{lx_{i\pm1/2}}^{n}$, written as `T_min_half` or `T_plus_half`

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 845 (delta 49), reused 0 (delta 0), pack-reused 696
Receiving objects: 100% (845/845), 4.95 MiB | 30.33 MiB/s, done.
Resolving deltas: 100% (347/347), done.


## Example 5.8 

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

In [2]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
pi = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 10 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor

1.1883597883597885

Because of uniform gridblocks, the equation for `tr_plus_coupling` and `tr_min_coupling` become:

$$T_{lx_{i+1/2}}^{n} = T_{lx_{i-1/2}}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i+1/2}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i-1/2}^{n}$$

In [3]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_x * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.1267875, 0.1267875)

There are 5 grids (`grid_1`, `grid_2`, `grid_3`, `grid_4`, `grid_5`), so the transmissibility coefficients will be: $$T_{lx_{i+1/2}}^{n}=(T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n}, T_{lx_{5+1/2}}^{n})$$

and

$$T_{lx_{i-1/2}}^{n}=(T_{lx_{1/2}}^{n}, T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n})$$

The values of $T_{lx_{5+1/2}}^{n}=0$ and $T_{lx_{1/2}}^{n}=0$ (the edge of gridblocks), so the values of each are:

$$T_{lx_{i+1/2}}^{n}=(0, 0.127, 0.127, 0.127, 0.127)$$

and

$$T_{lx_{i-1/2}}^{n}=(0.127, 0.127, 0.127, 0.127, 0)$$




In [4]:
p = np.full(ngrid-1, T_plus_half)
Ti_plus_halves = np.append([0], p)
print(Ti_plus_halves)
q = np.full(ngrid-1, T_min_half)
Ti_min_halves = np.append(q, [0])
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.        0.1267875 0.1267875 0.1267875 0.1267875]
[0.1267875 0.1267875 0.1267875 0.1267875 0.       ]
At grid 1, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.0 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.1267875 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.1267875 for T_plus_half.


Requires array for $q_{sc}$

In [0]:
qsc = [0, 0, 0, -150, 0] # production well in grid 4

Calculate $p_{i}^{n+1}$ for each grid in each time

In [6]:
pi = 6000
for i in range(ngrid):
  pi = pi + (factor * ((Ti_plus_halves[i] * pi) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi) + (Ti_min_halves[i] * pi)) + (factor * qsc[i]))
  print(pi)

6000.0
6000.0
6000.0
5821.746031746032
5821.746031746032


In [7]:
pi = [6000, 6000, 6000, 6000, 6000]
qsc = [0, 0, 0, -150, 0]
minus = pi[0]
plus = pi[-1]

pnew_arr = []
for i, obj in enumerate(pi):
  if i > 0:
    minus = pi[i-1]
  if i < (len(pi) - 1):
    plus = pi[i+1]

  pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
  
  pnew_arr.append(float(pnew))
pnew_arr

[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]

In [8]:
pi = [6000, 6000, 6000, 6000, 6000]
qsc = [0, 0, 0, -150, 0]
minus = pi[0]
plus = pi[-1]
time = [10, 20, 30, 40, 50]

pi_arr = []
for j in range(len(time)):
  pnew_arr = []

  # minus = pi[0]
  # plus = pi[-1]

  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))
  pi = pnew_arr
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df

,0,1,2,3,4
0,6000.000000,6000.000000,6000.000000,5821.746032,6000.0
1,6000.000000,6000.000000,5973.142623,5697.206817,6000.0
2,5973.142623,5995.953421,5935.614184,5606.149463,6000.0
3,5931.567605,5983.425275,5895.065272,5536.876802,6000.0
4,5882.537125,5962.298795,5854.410441,5482.369178,6000.0


In [0]:
data = [['New York Yankees', 'Acevedo Juan', 900000, 'Pitcher'], 
        ['New York Yankees', 'Anderson Jason', 300000, 'Pitcher'], 
        ['New York Yankees', 'Clemens Roger', 10100000, 'Pitcher'], 
        ['New York Yankees', 'Contreras Jose', 5500000, 'Pitcher']]

df = pd.DataFrame.from_records(data)
df

,0,1,2,3
0,New York Yankees,Acevedo Juan,900000,Pitcher
1,New York Yankees,Anderson Jason,300000,Pitcher
2,New York Yankees,Clemens Roger,10100000,Pitcher
3,New York Yankees,Contreras Jose,5500000,Pitcher


# ***

In [0]:
pi = [200, 300, 500, 700, 800]
foo = 800

def minplus(objects, foo):
  # get rid of boundary problem
  minus = objects[0]
  plus = objects[-1]
  
  l = len(objects)
  for index, obj in enumerate(objects):
      if obj == foo:
          if index > 0:
              minus = objects[index - 1]
          if index < (l - 1):
              plus = objects[index + 1] 
  return(minus, plus)

minus, plus = minplus(pi, foo)
print(minus)
print(plus)

700
800


In [0]:
time = [10, 20, 30, 40, 50]
pnewww_arr = []
for j in range(len(time)):
  pi = [6000, 6000, 6000, 6000, 6000]
  pnew_arr =[]
  for i in range(ngrid):
    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * pi[i]) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * pi[i])) + (factor * qsc[i]))
    pnew_arr.append(float(pnew))
  pi = pnew_arr
  print(pi)

[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]
[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]
[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]
[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]
[6000.0, 6000.0, 6000.0, 5821.746031746032, 6000.0]


In [0]:
a = [1, 4, 5, 7, 9]

for i, n in enumerate(a):
  if i < (len(a)):
    caw = a[i]- a[i-1]
    print(caw)

-8
3
1
2
2


In [0]:
pi = [1000, 2000, 3000, 4000, 5000]
for i, n in enumerate(pi):
  if i < (len(pi)):
    c = pi[i - 1]
    print(c)


5000
1000
2000
3000
4000


In [0]:
pi = [1, 2, 3, 4, 5]

for i, string in enumerate(pi):
  c = pi[i] * pi[i-1]
  print(c)

5
2
6
12
20


In [0]:
pi = 6000
for i in range(ngrid):
  pi = pi + (factor * ((Ti_plus_halves[i] * pi) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi) + (Ti_min_halves[i] * pi)) + (factor * qsc[i]))
  print(pi)

In [0]:
def find_prev_next(elem, elements):
    previous, next = None, None
    index = elements.index(elem)
    if index > 0:
        previous = elements[index -1]
    if index < (len(elements)-1):
        next = elements[index +1]
    return previous, next

# listi = [1000,2000,3000,4000,5000,6000]
listi = [6000,6000,6000,6000,5000,6000]
foo = listi[2]

previous, nexti = find_prev_next(foo, listi)

print(previous) # should print 'two'
print(nexti) # should print 'four'

None
6000
